In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:15pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

In [2]:
import requests #http요청을 보내기 위한 라이브러리
import urllib.parse #url관련 작업을 위한 표준 라이브러리(인코딩, 디코딩, 파싱 등)
from datetime import datetime, timedelta # datetime: 특정 날짜, 시간 조작 / timedelta:시간 간격, 날짜 계산
import xml.etree.ElementTree as ET #XML 문서를 파싱하고 조작하기 위한 표준 라이브러리(읽기, 파싱, 수정)

In [ ]:
class MultiWeatherAPI:
    """4개 기상청 API 통합 관리 클래스"""
    def __init__(self, service_key):
        self.service_key = urllib.parse.unquote(service_key) #service_key를 원래 형태로 디코딩
        
        self.apis = { #4개 api 이름:{정보} 딕셔너리
            'asos_hourly': { #기상청_지상(종관, ASOS) 시간자료 조회서비스
                'url': 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList',
                'name': '지상(ASOS) 시간자료',
                'description': '실시간 관측소 데이터 (정확한 강수량, 온도, 습도)'
            },
            'asos_daily': { #기상청_지상(종관, ASOS) 일자료 조회서비스
                'url': 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList', 
                'name': '지상(ASOS) 일자료',
                'description': '일별 종합 기상 데이터 (누적 강수량, 최고/최저 온도)'
            },
            'weather_warning': { #기상청_기상특보 조회서비스
                'url': 'http://apis.data.go.kr/1360000/WthrWrnInfoService/getWthrWrnList',
                'name': '기상특보',
                'description': '호우경보, 대설경보 등 기상특보 (침수 위험 직접 지표)'
            },
            'short_forecast': { #기상청_단기예보 ((구)_동네예보) 조회서비스
                'url': 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst',
                'name': '단기예보(초단기실황)',
                'description': '격자 기반 실시간 데이터'
            }
        }
        
        self.seoul_stations = { #서울 관측소들 정보
            'main': {'stnId': '108', 'name': '서울', 'nx': 60, 'ny': 127},
            'gangnam': {'stnId': '401', 'name': '서울강남', 'nx': 61, 'ny': 125},
            'songpa': {'stnId': '402', 'name': '서울송파', 'nx': 62, 'ny': 126}
        }
    
    def get_comprehensive_weather_data(self): #4개의 api에서 데이터 수집하는 메서드
        print("🌤️ 4개 API 종합 데이터 수집 시작...")
        
        results = { #반환할 결과 형식
            'timestamp': datetime.now(), #리턴 당시 시간
            'data_sources': [], #데이터를 어디서 가져왔는지 기록
            'weather_data': {}, #실제 날씨 정보
            'warnings': [], #경고 메시지들
            'forecast': {}, #날씨 예보 정보
            'success': False, #작업 성공 여부
            'errors': [] #오류 메시지들
        }
        
        # 1. ASOS 시간자료 (가장 정확한 실시간 데이터)
        # 2. ASOS 일자료 (누적/통계 데이터)
        # 3. 기상특보 (침수 직접 경보)
        # 4. 단기예보 (격자 데이터)
        
    def fetch_asos_hourly_data(self):
        #1번(시간자료) 수집